In [1]:
### 载入数据
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('./data/data_raw.csv')

In [4]:
data.shape

(4989, 33)

In [6]:
data.head(2)

,label,num,w0,w1,w2,w3,w4,w5,w6,h0,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,3,960,1599,1635,1620,1510.0,1534.0,482.0,416.0,216.0,...,2722.0,2594.0,2483.0,2006.0,1236.0,881.0,663.0,589.0,464.0,279.0
1,4,13673,5308,5195,4890,4792.0,6099.0,4932.0,5606.0,270.0,...,6804.0,5405.0,5031.0,4900.0,3342.0,2622.0,2081.0,1457.0,978.0,475.0


In [5]:
#data.apply(lambda x: sum(x.isnull()))

In [6]:
data.fillna(-999,inplace=True)      #缺失值填充

In [7]:
#data.apply(lambda x: sum(x.isnull()))

In [8]:
data.head(2)

,label,num,w0,w1,w2,w3,w4,w5,w6,h0,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
0,3,960,1599,1635,1620,1510.0,1534.0,482.0,416.0,216.0,...,2722.0,2594.0,2483.0,2006.0,1236.0,881.0,663.0,589.0,464.0,279.0
1,4,13673,5308,5195,4890,4792.0,6099.0,4932.0,5606.0,270.0,...,6804.0,5405.0,5031.0,4900.0,3342.0,2622.0,2081.0,1457.0,978.0,475.0


In [9]:
target=data['label']

In [10]:
data.drop('label',axis=1,inplace=True)

In [11]:
### 数据分割
from sklearn.model_selection import train_test_split                 # 载入数据分割函数train_test_split
x_train,x_test,y_train,y_test = train_test_split(data,        # 特征空间
                                                 target,      # 输出空间
                                                 test_size = 0.1,    # 测试集占30%
                                                 random_state = 33)  # 为了复现实验，设置一个随机数

In [12]:
x_train.shape,y_train.shape

((4490, 32), (4490,))

In [13]:
x_test.shape,y_test.shape

((499, 32), (499,))

In [14]:
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import explained_variance_score
import xgboost as xgb
import xgboost as xgb
import pandas as pd
import numpy as np

/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This m

In [17]:
### 模型相关
from xgboost import XGBClassifier
#model = XGBClassifier()               # 载入模型（模型命名为model)
model=XGBClassifier(
        learning_rate =0.1,
        n_estimators=3000, 
        max_depth=5,
        min_child_weight=1, 
        gamma=0.2, 
        subsample=0.7,
        colsample_bytree=0.9,
        objective= 'binary:logistic',
        nthread=4, 
        scale_pos_weight=1,
        seed=27
        )
model.fit(x_train,y_train)            # 训练模型（训练集）
y_pred = model.predict(x_test)        # 模型预测（测试集），y_pred为预测结果
print('Finished')

Finished


/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
### 性能度量
from sklearn.metrics import accuracy_score   # 准确率
accuracy = accuracy_score(y_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))


accuarcy: 51.70%


In [34]:
### 特征重要性
import matplotlib.pyplot as plt
from xgboost import plot_importance
fig,ax = plt.subplots(figsize=(10,15))
plot_importance(model,height=0.5,max_num_features=12,ax=ax)
plt.show()

In [35]:
#对subsample 和 max_features 用grid search查找最好的参数
param_test1 = {
    'max_depth':[3,5,7,9,11],
    'min_child_weight':[1,3,5,7]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=500, max_depth=5,
                                        min_child_weight=1, gamma=0.2, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1,n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_train,y_train)

/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning

/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning

/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning

/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/qyl/anaconda3/envs/DL_py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.2, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [3, 5, 7, 9, 11], 'min_child_weight': [1, 3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [38]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.51714, std: 0.01126, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.51714, std: 0.01889, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.51781, std: 0.01367, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.51669, std: 0.01376, params: {'max_depth': 3, 'min_child_weight': 7},
  mean: 0.52425, std: 0.01271, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.51892, std: 0.01570, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.51892, std: 0.01351, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.51268, std: 0.01039, params: {'max_depth': 5, 'min_child_weight': 7},
  mean: 0.52046, std: 0.01624, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.51781, std: 0.00945, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.52182, std: 0.00887, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.51535, std: 0.01395, params: {'max_depth': 7, 'min_child_weight': 7},
  mean: 0.52001, std: 0.01219, params: {